In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import random
import glin.ranking as ranking

In [3]:
# Create fake data

def fake_data():
    # 11 elements
    nElements = 11

    blobId =  [1,  1,  1,  2,  2,  3,  3,  3,  3,  4,  4]
    routeId = [10, 11, 12, 21, 22, 31, 32, 33, 34, 41, 42]

    random.seed(0)
    railDistance = np.array([random.randint(10,200) for x in range(nElements)])
    random.seed(1)
    roadDistance = np.array([random.randint(1,200) for x in range(nElements)])
    random.seed(2)
    portDistance = np.array([random.randint(50,200) for x in range(nElements)])
    random.seed(3)
    airDistance =  np.array([random.randint(100,200) for x in range(nElements)])

    random.seed(10)
    railDistance[random.sample(range(nElements), 5)] = 0
    random.seed(11)
    roadDistance[random.sample(range(nElements), 2)] = 0
    random.seed(12)
    portDistance[random.sample(range(nElements), 8)] = 0
    random.seed(13)
    airDistance[random.sample(range(nElements), 8)]  = 0

    totalDistance = railDistance + roadDistance + portDistance + airDistance

    # Calling DataFrame constructor after zipping
    # both lists, with columns specified
    return pd.DataFrame(list(zip(blobId, routeId, totalDistance, railDistance, roadDistance, portDistance, airDistance)),
                columns =['BlobId', 'RouteId', 'TotalDistance','RailDistance','RoadDistance','PortDistance','AirDistance'])

def make_weights():
    # less is better, and more carbon should be storngly punished
    upperCarbonApprox = -20
    # less is better, and time is more important than distance itself
    timeApprox = -5 
    # less is better
    railDistance = -1
    # less is better
    roadDistance = -2
    # less is better
    portDistance = -1
    # less is better
    airDistance = -3

    # it's good to use rail
    usesRail = 3
    # it's good to use ships
    usesPort = 1
    # it's bad to use planes
    usesAir = -10

    return {
        'UpperCarbonApprox':upperCarbonApprox,
        'TimeApprox':timeApprox,
        'RailDistance':railDistance,
        'RoadDistance':roadDistance,
        'PortDistance':portDistance,
        'AirDistance':airDistance,
        'UsesRail':usesRail,
        'UsesPort':usesPort,
        'UsesAir':usesAir
    }

In [4]:
# Process data

routes = fake_data()
weights = make_weights()

processed = ranking.do_preprocesing(routes)
ranking.calculate_score(processed, weights)

processed.sort_values("Score", ascending=False)


,BlobId,RouteId,TotalDistance,RailDistance,RoadDistance,PortDistance,AirDistance,LowerCarbonApprox,UpperCarbonApprox,TimeApprox,UsesRail,UsesAir,UsesPort,Score
3,2,21,93,76,17,0,0,3300,10150,1.550000,True,False,False,-25.014423
4,2,22,66,0,66,0,0,3960,9900,1.100000,False,False,False,-25.817308
5,3,31,165,134,31,0,0,5880,18050,2.750000,True,False,False,-31.221154
2,1,12,216,20,196,0,0,12360,31400,3.600000,True,False,False,-35.990385
1,1,11,263,117,146,0,0,12270,33600,4.383333,True,False,False,-39.899840
6,3,32,241,0,127,114,0,8760,23610,4.016667,False,False,True,-39.967949
0,1,10,165,0,35,0,130,67100,70250,2.750000,False,True,False,-44.855769
7,3,33,180,0,0,0,180,90000,90000,3.000000,False,True,False,-46.298077
9,4,41,319,0,121,198,0,9240,26070,5.316667,False,False,True,-46.641026
10,4,42,416,159,167,90,0,15690,44550,6.933333,True,False,True,-52.068109
